In [ ]:
! nvidia-smi

In [ ]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
t = TicToc()

In [ ]:
mainVol = np.load('Your Path Here/normalizeddata.npy')

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data).float()

        
    def __getitem__(self, index):
        x = self.data[index]
        #x = process_scan(x)
        
        return x
    
    def __len__(self):
        return len(self.data)

trainData = MyDataset(mainVol)
train_loader = DataLoader(trainData,batch_size=10)  

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=131072):
        return input.view(input.size(0), 64, 16, 16, 8)

In [ ]:
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True,
                 bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

        self.bn = nn.BatchNorm3d(out_planes, eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        # print('conv block output',x.shape)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Autoencoders(nn.Module):
    def __init__(self):
        super(Autoencoders, self).__init__()

        self.relu = nn.ReLU(True)

        ##encoder layers
        self.conv_block_1 = BasicConv(1, 16, 3, stride=1, padding=1, bn=True, relu=True)
        self.conv_block_2 = BasicConv(16, 32, 3, stride=1, padding=1, bn=True, relu=True)
        self.conv_block_3 = BasicConv(32, 64, 3, stride=1, padding=1, bn=True, relu=True)
        self.max_pool = nn.MaxPool3d(2)
        self.flatten = Flatten()
        self.enclinear = nn.Linear(131072, 1024)
        ##decoder layers
        self.dec_convtrans_1 = nn.ConvTranspose3d(64, 32, 2, stride=2)
        self.dec_convtrans_2 = nn.ConvTranspose3d(32, 16, 2, stride=2)
        self.dec_convtrans_3 = nn.ConvTranspose3d(16, 1, 2, stride=2)
        self.deconv_batch_norm_1 = nn.BatchNorm3d(32)
        self.deconv_batch_norm_2 = nn.BatchNorm3d(16)
        self.unflatten = UnFlatten()
        self.declinear = nn.Linear(1024, 131072)

    def forward(self, x):
        # Encoder
        code1 = self.conv_block_1(x)
        # print("code1:", code1.shape)
        code1 = self.max_pool(code1)
        # print("code1:", code1.shape)
        code2 = self.conv_block_2(code1)
        # print("code2:", code2.shape)
        code2 = self.max_pool(code2)
        # print("code2:", code2.shape)
        code3 = self.conv_block_3(code2)
        # print("code3:", code3.shape)
        code3 = self.max_pool(code3)
        #print("code3:", code3.shape)
        code3 = self.flatten(code3)
        # print("code3:", code3.shape)
        code3 = self.enclinear(code3)
        # print("code3:", code3.shape)

        # Decoder
        out1 = self.declinear(code3)
        out1 = self.unflatten(out1)
        out1 = self.dec_convtrans_1(out1)
        # print("out1:", out1.shape)
        out1 = self.deconv_batch_norm_1(out1)
        # print("out1:", out1.shape)
        out2 = self.dec_convtrans_2(out1)
        # print("out2:", out2.shape)
        out2 = self.deconv_batch_norm_2(out2)
        # print("out2:", out2.shape)
        out3 = self.dec_convtrans_3(out2)
        # print("out3:", out3.shape)

        return code3, out3

In [ ]:
model = Autoencoders()
model = model.to(device)
print(model)

In [ ]:
model = torch.load('/home/norouzi1/Thesis_FinalResults/Covid/SavedModels/Autoencoder.pth')
model.eval()

In [ ]:
enc_output = np.loadtxt('/home/norouzi1/Thesis_FinalResults/Covid/SavedModels/enc10.txt')
enc_output = enc_output.astype(int)

In [ ]:
labels = np.loadtxt('/home/norouzi1/Thesis_FinalResults/Covid/SavedModels/labels.txt')
labels = labels.astype(int)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.from_numpy(data).float()
        self.labels = labels.long()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

labels = torch.from_numpy(labels)
trainData = MyDataset(mainVol, labels)
train_loader = DataLoader(trainData,batch_size=10)  

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class ClassificationModel(nn.Module):   
    def __init__(self):
        super(ClassificationModel, self).__init__()

        self.features = nn.Sequential(
            # Defining a 3D convolution layer
            nn.Conv3d(1,64,3, stride=1, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2)),
            # Defining another 3D convolution layer
            nn.Conv3d(64,32,3, stride=1, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2)),
            # Defining another 3D convolution layer
            nn.Conv3d(32,16,3, stride=1, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2))
        )

        self.classifier = nn.Sequential(
            Flatten(),
            nn.Linear(32768, 256),
            nn.ReLU(inplace=True),
            #nn.Dropout3d(p=0.5),
            nn.Linear(256, 3)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
model = ClassificationModel()
model = model.to(device)
print(model)

In [ ]:
model = torch.load('Your Path Here/Classifier.pth')

In [ ]:
class GradCAMModel(nn.Module):
    def __init__(self):
        super(GradCAMModel, self).__init__()
        
        # get the classification network
        self.gradcam = model
        
        # disect the network to access its last convolutional layer
        self.features_conv = self.gradcam.features[:11]
        
        # get the max pool of the features stem
        self.max_pool = nn.MaxPool3d((2,2,2))
        
        # get the classifier of the model
        self.classifier = self.gradcam.classifier
        
        # placeholder for the gradients
        self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.features_conv(x)
        
        # register the hook
        h = x.register_hook(self.activations_hook)
        
        # apply the remaining pooling
        x = self.max_pool(x)
        x = self.classifier(x)
        return x
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        return self.features_conv(x)

In [ ]:
model2 = GradCAMModel()
model2 = model2.to(device)

In [ ]:
model2 = torch.load('Your Path Here/GradCAM.pth')

In [ ]:
def resize_heat(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.from_numpy(data).float()
        self.labels = labels.long()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        
        return index, x
    
    def __len__(self):
        return len(self.data)

trainData = MyDataset(mainVol, labels)
train_loader = DataLoader(trainData,batch_size=1)  

In [ ]:
def get_distance_matrix( X_trn, X_tst):
  yTy = np.sum( np.square( X_trn) , axis=1)
  xTy = np.dot(X_trn, np.transpose(X_tst))
  yTy = yTy.reshape(yTy.size, 1)
  dist_matrix = yTy - 2*xTy
  return dist_matrix

In [ ]:
sum_mems_0 = []
imgs_cl0 = []
indx_0 = []
num_mems_0 = 0
for index, data in train_loader:
    if labels[index] == 0:
        num_mems_0 += 1
        indx_0.append(index)
        data = data.unsqueeze(1)
        # get the image from the dataloader
        main_img = data[0,...]
        main_img = main_img.cuda().detach().cpu().numpy()
        imgs_cl0.append(main_img)  
        img = data.to(device=device, dtype=torch.float) 
        winning_idx = model2(img).argmax(dim=1)
        pred = model2(img)
        # get the gradient of the output with respect to the parameters of the model
        pred[:,winning_idx].backward()
        # pull the gradients out of the model
        gradients = model2.get_activations_gradient()
        #print(gradients.shape)
        # pool the gradients across the channels
        alpha_ks = torch.mean(gradients, dim=[0, 2, 3, 4])
        #print(pooled_gradients.shape)
        # get the activations of the last convolutional layer
        weighted_combination = model2.get_activations(img).detach()
        #print(activations.shape)
        # weight the channels by corresponding gradients
        for i in range(16):
            weighted_combination[:, i, :, :, :] *= alpha_ks[i]

        # average the channels of the activations
        heatmap = torch.sum(weighted_combination, dim=1).squeeze()
        heatmap = heatmap.detach().cpu().numpy()
        heatmap = np.maximum(heatmap, 0)
        probability = pred[:,winning_idx].cuda().detach().cpu().numpy()
        heatmap *= probability
        heatmap = torch.from_numpy(heatmap)
        heatmap = heatmap.cuda().detach().cpu()
        heatmap_resized = resize_heat(heatmap)
        sum_mems_0.append(heatmap_resized)       

sum_mems_0 = np.stack(sum_mems_0)
sum_mems_0 /= num_mems_0
imgs_cl0 = np.stack(imgs_cl0)
indx_0 = np.stack(indx_0)

In [ ]:
img_means = np.mean(imgs_cl0, 0)
print(img_means.shape)
resh_vol = np.reshape( imgs_cl0, [imgs_cl0.shape[0],-1])
print(resh_vol.shape)
resh_mean = np.reshape( img_means, [1,-1])
print(resh_mean.shape)
dist_matrix = get_distance_matrix(resh_vol, resh_mean)
print(dist_matrix.shape)

# initialize K
K = 10
  
# Smallest K elements indices
# using sorted() + lambda + list slicing
res = sorted(range(len(dist_matrix)), key = lambda sub: dist_matrix[sub])[:K]

closest_to_mean_0 = [indx_0[i] for i in res]

In [ ]:
along_datapoints_0 = np.mean(sum_mems_0, axis=0)
print(along_datapoints_0.shape)

In [ ]:
np.argwhere(np.isnan(sum_mems_0))

In [ ]:
main_img_avg_cl0 = np.mean(imgs_cl0, axis=0)
main_img_avg_cl0 = main_img_avg_cl0[0,:,:,:]
superimposed_img_0 = along_datapoints_0 + main_img_avg_cl0

In [ ]:
along_z = np.mean(along_datapoints_0, axis=2)
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.imshow(along_z)
ax = plt.gca()
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
along_xy = along_datapoints_0.mean(axis=(0, 1))
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.xlabel("Slice Depth", fontsize=250, color = 'white')
plt.ylabel("VolPAM_1D", fontsize=250, color = 'white')
plt.plot(along_xy, color = "mediumvioletred") # plotting by columns
ax = plt.gca()
ax.set_facecolor('midnightblue')
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
sum_mems_1 = []
imgs_cl1 = []
indx_1 = []
num_mems_1 = 0
for index, data in train_loader:
    if labels[index] == 1:
        num_mems_1 += 1
        indx_1.append(index)
        data = data.unsqueeze(1)
        # get the image from the dataloader
        main_img = data[0,...]
        main_img = main_img.cuda().detach().cpu().numpy()
        imgs_cl1.append(main_img)  
        img = data.to(device=device, dtype=torch.float) 
        winning_idx = model2(img).argmax(dim=1)
        pred = model2(img)

        # get the gradient of the output with respect to the parameters of the model
        pred[:,winning_idx].backward()

        # pull the gradients out of the model
        gradients = model2.get_activations_gradient()
        #print(gradients.shape)
        # pool the gradients across the channels
        alpha_ks = torch.mean(gradients, dim=[0, 2, 3, 4])
        #print(pooled_gradients.shape)
        # get the activations of the last convolutional layer
        weighted_combination = model2.get_activations(img).detach()
        #print(activations.shape)
        # weight the channels by corresponding gradients
        for i in range(16):
            weighted_combination[:, i, :, :, :] *= alpha_ks[i]

        # average the channels of the activations
        heatmap = torch.sum(weighted_combination, dim=1).squeeze()
        heatmap = heatmap.detach().cpu().numpy()
        heatmap = np.maximum(heatmap, 0)
        probability = pred[:,winning_idx].cuda().detach().cpu().numpy()
        heatmap *= probability
        heatmap = torch.from_numpy(heatmap)
        heatmap = heatmap.cuda().detach().cpu()
        heatmap_resized = resize_heat(heatmap)
        sum_mems_1.append(heatmap_resized)       
        
sum_mems_1 = np.stack(sum_mems_1)
sum_mems_1 /= num_mems_1
imgs_cl1 = np.stack(imgs_cl1)
indx_1 = np.stack(indx_1)

In [ ]:
img_means = np.mean(imgs_cl1, 0)
print(img_means.shape)
resh_vol = np.reshape( imgs_cl1, [imgs_cl1.shape[0],-1])
print(resh_vol.shape)
resh_mean = np.reshape( img_means, [1,-1])
print(resh_mean.shape)
dist_matrix = get_distance_matrix(resh_vol, resh_mean)
print(dist_matrix.shape)

# initialize K
K = 10
  
# Smallest K elements indices
# using sorted() + lambda + list slicing
res = sorted(range(len(dist_matrix)), key = lambda sub: dist_matrix[sub])[:K]

closest_to_mean_1 = [indx_1[i] for i in res]

In [ ]:
along_datapoints_1 = np.mean(sum_mems_1, axis=0)
print(along_datapoints_1.shape)

In [ ]:
main_img_avg_cl1 = np.mean(imgs_cl1, axis=0)
main_img_avg_cl1 = main_img_avg_cl1[0,:,:,:]
superimposed_img_1 = along_datapoints_1 + main_img_avg_cl1

In [ ]:
along_z = np.mean(along_datapoints_1, axis=2)
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.imshow(along_z)
ax = plt.gca()
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
along_xy = along_datapoints_1.mean(axis=(0, 1))
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.xlabel("Slice Depth", fontsize=250, color = 'white')
plt.ylabel("VolPAM_1D", fontsize=250, color = 'white')
plt.plot(along_xy, color = "mediumvioletred") # plotting by columns
ax = plt.gca()
ax.set_facecolor('midnightblue')
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
sum_mems_2 = []
imgs_cl2 = []
indx_2 = []
num_mems_2 = 0
for index, data in train_loader:
    if labels[index] == 2:
        indx_2.append(index) 
        num_mems_2 += 1
        data = data.unsqueeze(1)
        # get the image from the dataloader
        main_img = data[0,...]
        main_img = main_img.cuda().detach().cpu().numpy()
        imgs_cl2.append(main_img)  
        img = data.to(device=device, dtype=torch.float) 
        winning_idx = model2(img).argmax(dim=1)
        pred = model2(img)

        # get the gradient of the output with respect to the parameters of the model
        pred[:,winning_idx].backward()

        # pull the gradients out of the model
        gradients = model2.get_activations_gradient()
        #print(gradients.shape)
        # pool the gradients across the channels
        alpha_ks = torch.mean(gradients, dim=[0, 2, 3, 4])
        #print(pooled_gradients.shape)
        # get the activations of the last convolutional layer
        weighted_combination = model2.get_activations(img).detach()
        #print(activations.shape)
        # weight the channels by corresponding gradients
        for i in range(16):
            weighted_combination[:, i, :, :, :] *= alpha_ks[i]

        # average the channels of the activations
        heatmap = torch.sum(weighted_combination, dim=1).squeeze()
        heatmap = heatmap.detach().cpu().numpy()
        heatmap = np.maximum(heatmap, 0)
        probability = pred[:,winning_idx].cuda().detach().cpu().numpy()
        heatmap *= probability
        heatmap = torch.from_numpy(heatmap)
        heatmap = heatmap.cuda().detach().cpu()
        heatmap_resized = resize_heat(heatmap)
        sum_mems_2.append(heatmap_resized)       
        
sum_mems_2 = np.stack(sum_mems_2)
sum_mems_2 /= num_mems_2
imgs_cl2 = np.stack(imgs_cl2)
indx_2 = np.stack(indx_2)

In [ ]:
img_means = np.mean(imgs_cl2, 0)
print(img_means.shape)
resh_vol = np.reshape( imgs_cl2, [imgs_cl2.shape[0],-1])
print(resh_vol.shape)
resh_mean = np.reshape( img_means, [1,-1])
print(resh_mean.shape)
dist_matrix = get_distance_matrix(resh_vol, resh_mean)
print(dist_matrix.shape)

# initialize K
K = 10
  
# Smallest K elements indices
# using sorted() + lambda + list slicing
res = sorted(range(len(dist_matrix)), key = lambda sub: dist_matrix[sub])[:K]

closest_to_mean_2 = [indx_2[i] for i in res]

In [ ]:
along_datapoints_2 = np.mean(sum_mems_2, axis=0)
print(along_datapoints_2.shape)

In [ ]:
main_img_avg_cl2 = np.mean(imgs_cl2, axis=0)
main_img_avg_cl2 = main_img_avg_cl2[0,:,:,:]
superimposed_img_2 = along_datapoints_2 + main_img_avg_cl2

In [ ]:
along_z = np.mean(along_datapoints_2, axis=2)
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.imshow(along_z)
ax = plt.gca()
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
along_xy = along_datapoints_2.mean(axis=(0, 1))
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.xlabel("Slice Depth", fontsize=250, color = 'white')
plt.ylabel("VolPAM_1D", fontsize=250, color = 'white')
plt.plot(along_xy, color = "mediumvioletred") # plotting by columns
ax = plt.gca()
ax.set_facecolor('midnightblue')
ax.tick_params(axis='x', labelsize=200, colors = 'white')
ax.tick_params(axis='y', labelsize=200, colors = 'white')

In [ ]:
close0 = [mainVol[i] for i in closest_to_mean_0]
close1 = [mainVol[i] for i in closest_to_mean_1]
close2 = [mainVol[i] for i in closest_to_mean_2]

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()

In [ ]:
for i in range(np.shape(close0)[0]):
    plot_slices(4, 16, 128, 128, np.array(close0)[i,:,:,:])
    plt.show() 

In [ ]:
for i in range(np.shape(close1)[0]):
    plot_slices(4, 16, 128, 128, np.array(close1)[i,:,:,:])
    plt.show() 

In [ ]:
for i in range(np.shape(close2)[0]):
    plot_slices(4, 16, 128, 128, np.array(close2)[i,:,:,:])
    plt.show() 

In [ ]:
# collect arrays in dictionary
savedict = {
    'A' : superimposed_img_0,
    'B' : superimposed_img_1,
    'C' : superimposed_img_2
}

savemat("Your Path Here/3DVis_IRANData.mat", savedict)